In [1]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob

In [4]:
# Loop over all files

filenames = glob.glob("/home/raulteixeira/scratch-midway2/dr3_1_1_bdf*")

In [5]:
filenames

['/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000020.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000021.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000010.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000002.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000040.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000001.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000034.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000006.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000026.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000037.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000018.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000023.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000036.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000044.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000017.h5',
 '/home/raulteixeira/scratch-midway2/dr3_1_1_bdf_000038.h5',
 '/home/raulteixeira/scr

In [3]:
def get_dataframes():
    all_dataframes = []
    for filename in filenames:
        # Open catalogs
        all_dataframes.append(pd.read_hdf(filename, key='data/table', mode='r'))

    data_df = pd.concat(all_dataframes)
    
    return data_df

In [4]:
# Open catalogs
data_df = get_dataframes()

specz = pd.read_table("/project2/chihway/delve_shear/BRPORTAL_E_6315_18670.csv", sep=',')

In [5]:
# Match spectra to DELVE catalog
spec_cd = SkyCoord(ra=specz['RA'].values*u.degree, dec=specz['DEC'].values*u.degree)
delve_cd = SkyCoord(ra=data_df['RA'].values*u.deg, dec=data_df['DEC'].values*u.deg)
idx, d2d, d3d = delve_cd.match_to_catalog_sky(spec_cd)
good_matches = d2d < 1.0*u.arcsec

In [6]:
print(len(np.unique(idx[good_matches])), np.count_nonzero(good_matches))

523813 524026


In [7]:
# Add spectra to DELVE catalog
data_df['zspec'] = np.nan
data_df.loc[good_matches, 'zspec'] = specz.iloc[idx[good_matches], specz.columns.get_loc('Z')].values

In [8]:
# Save each tile separately
unique_tilenames = np.unique(data_df.TILENAME)
unique_tilenames.shape

(6684,)

In [ ]:
for tilename in unique_tilenames:
    data_df_sub = data_df[data_df.TILENAME==tilename]
    data_df_sub.to_csv('/home/raulteixeira/scratch-midway2/CosmicShearData/spectiles/'+tilename+'.csv.gz', sep=',', header=True, index=False)

In [1]:
# Save one file containing the subset with matched spectra.
data_df_spec = data_df[np.isfinite(data_df.zspec.values)]
data_df_spec.to_csv('/home/raulteixeira/scratch-midway2/CosmicShearData/DELVE_data_with_zspec012623.csv.gz', sep=',', header=True, index=False)